# Ragasを使ってテストデータを生成する

In [1]:
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv(override=True)

True

ドキュメントを読み込みます

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.text import TextLoader

loader = DirectoryLoader("./about-aws/whats-new/2024/", loader_cls=TextLoader)
documents = loader.load()

for document in documents:
    document.metadata["filename"] = document.metadata["source"]

In [3]:
documents[0]

Document(metadata={'source': 'about-aws/whats-new/2024/04/amazon-neptune-connector-nodestream-parquet-sbom.json', 'filename': 'about-aws/whats-new/2024/04/amazon-neptune-connector-nodestream-parquet-sbom.json'}, page_content='{\n  "title": "Nodestream 用の Amazon Neptune コネクタ、Parquet、SBOM に関する発表",\n  "body": "本日、Nodestream 用の Amazon Neptune コネクタ、Nodestream の Parquet ファイル形式による入力への対応、および CycloneDX と SPDX ファイル形式用の Nodestream Security Bill Of Material (SBOM) プラグインが利用可能になったことが発表されました。\\n\\nNodestream は、ETL (抽出、変換、ロード) のためのオープンソースプロジェクトで、柔軟性と拡張性を備えるように設計されており、データを収集したりグラフとしてモデル化する方法を定義したりできます。データの収集方法と処理方法を定義するためにパイプラインベースのアプローチを使用しており、スキーマ変更時のグラフの更新方法を定義できます。\\n\\nSBOM は、組織がソフトウェアアプリケーションの透明性、セキュリティ、信頼性を向上させるのに役立ちます。Nodestream SBOM プラグインによって、SBOM データ分析用の独自のグラフデータモデルが提供されます。SBOM は、サイバーリスク軽減のための高度なサプライチェーン機能を提供するフルスタックの Bill of Materials (BOM) 標準である CycloneDX、および Linux 用の System Package Data Exchange (SPDX) からインポートされます。SBOM は、GitHub と Amazon Inspector からもインポートされます。\\n\\nNodestream、Amazon Neptun

LangChainのモデルを定義します。

評価対象のLLMと違うモデルを使用したほうが公平な評価になると思い、Azure OpenAI Serviceのモデルを使用しています。

.env.sampleに、を使用する場合のを参考に、.envファイルを作成してから実行してください。

In [4]:
import os

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

generator_llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_GPT4OMINI", None)
)
critic_llm = AzureChatOpenAI(azure_deployment=os.getenv("AZURE_DEPLOYMENT_GPT4O", None))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_EMBEDDINGS", None)
)

RagasのTestsetGeneratorを生成します

In [5]:
from ragas.run_config import RunConfig
from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import TestsetGenerator

run_config = RunConfig(max_wait=600, max_retries=100)

generator = TestsetGenerator.from_langchain(
    generator_llm, critic_llm, embeddings, run_config=run_config
)

使用するプロンプトを日本語に合わせて最適化します。
（手作業でプロンプトを修正したファイルをcache_dirに用意しています）

In [6]:
language = "japanese"

generator.adapt(language, evolutions=[simple, reasoning, multi_context], cache_dir="./ragas/.cache")

生成を開始します。`test_size`が生成するテストデータの数です。

In [7]:
testset = generator.generate_with_langchain_docs(
    documents,
    test_size=10,  # 生成するテストデータの数
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
    raise_exceptions=False,
    run_config=run_config,
)

embedding nodes:   0%|          | 0/3082 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

生成したテストデータを確認します。

In [8]:
test_df = testset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,データサイエンティストはAmazon SageMaker Studioノートブックでどのよう...,"[{\n ""title"": ""Amazon SageMaker Studio ノートブック...",データサイエンティストは、Amazon SageMaker Studio の Jupyter...,simple,[{'source': 'about-aws/whats-new/2024/04/amazo...,True
1,フルマネージド型のデータベースインスタンスを作成するには、どのような手順を踏む必要がありますか?,[照してください。 [Amazon RDS マネジメントコンソール](https://con...,フルマネージド型の M7g または R7g データベースインスタンスを作成するには、Amaz...,simple,[{'source': 'about-aws/whats-new/2024/02/amazo...,True
2,AWS CloudShell でコンテナの起動はどのように簡単になったのでしょうか?,"[{\n ""title"": ""AWS CloudShell は 13 のリージョンで Do...",AWS CloudShell では Docker の組み込みサポートを開始したため、デベロッ...,simple,[{'source': 'about-aws/whats-new/2024/01/aws-c...,True
3,高い総スループットを実現するために、Mountpoint for Amazon S3 CSI...,"[{\n ""title"": ""Mountpoint for Amazon S3 CSI ド...",Mountpoint for Amazon S3 CSI ドライバーは、Bottlerock...,simple,[{'source': 'about-aws/whats-new/2024/03/mount...,True
4,Amazon Time Sync Serviceはどのようにして追加料金なしで使用できるのですか?,[タンスタイプにも拡大する予定です。このサービスは追加料金なしで使用できます。\n\nAma...,与えられた質問に対する答えは文脈の中に存在しない,simple,[{'source': 'about-aws/whats-new/2024/03/amazo...,True


テストデータを保存します。

In [9]:
test_df.to_json("testdataset.json", force_ascii=False)